# CDS Forecast Data Ingestor
This notebook ingests forecast data from the Climate Data Store (CDS) API and saves it in a zarr format.


# Dependecies



In [1]:
%%capture

import os
import json
import datetime
import requests
import getpass
import pprint

import xarray as xr

!pip install s3fs
import s3fs

!pip install "cdsapi>=0.7.4"
import cdsapi


# Parameters



In [1]:
# Forcast variables
variables = ['maximum_2m_temperature_in_the_last_24_hours']

# Bouning box of interest in format min_lon, min_lat, max_lon, max_lat
region = [10, 20, 30, 40]

# init forecast time "YYYY-MM"
init_time = datetime.datetime.strptime(2023-01-01, "%Y-%m-%d").replace(day=1)

# lead forecast time "YYYY-MM"
lead_time = datetime.datetime.strptime(2023-02-01, "%Y-%m-%d").replace(day=1)

# ingested data ouput zarr file
zarr_output = 'output.zarr'


# ICISK API call
This section exploits the I-CISK process _cds-ingestor_ to build data retrieval and storage.


In [1]:
# Create payload for ICisk API

icisk_api_payload = {
    "inputs": {
        "dataset": "seasonal-original-single-levels",
        "file_out": f"/tmp/{zarr_output.replace('.zarr', '')}.netcdf",
        "query": {
            "originating_centre": "ecmwf",
            "system": "51",
            "variable": ['maximum_2m_temperature_in_the_last_24_hours'],
            "year": [f"{init_time.year}"],
            "month": [f"{init_time.month:02d}"],
            "day": ["01"],
            "leadtime_hour": [str(h) for h in range(24, int(lead_time - init_time).total_seconds() // 3600, 24)],
            "area": [
                region[3],
                region[0],
                region[1],
                region[2]
            ],
            "data_format": "netcdf"
        },
        "token": "YOUR-ICISK-API-TOKEN",
        "zarr_out": f"s3://saferplaces.co/test/icisk/ai-agent/{zarr_output}",
    }
}

print('-------------------------------------------------------------------------------')

pprint.pprint(icisk_api_payload)

print('-------------------------------------------------------------------------------')

# Request Token and run the process

icisk_api_token = getpass.getpass("YOUR ICISK-API-TOKEN: ")

icisk_api_payload['inputs']['token'] = icisk_api_token

icisk_api_response = requests.post(
    url = 'https://i-cisk.dev.52north.org/ingest/processes/ingestor-cds-process/execution',
    json = icisk_api_payload
)

print('-------------------------------------------------------------------------------')

# Print the response

pprint.pprint({
    'status_code': icisk_api_response.status_code, 
    'response': icisk_api_response.json(),
})

print('-------------------------------------------------------------------------------')
